In [1]:
# useful packages
import numpy as np
import csv
import math
import pandas as pd
from sklearn import preprocessing
from sklearn import datasets
from sklearn import cluster
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import itertools

In [3]:
# load data as a dataframe with pandas
user_history = pd.read_csv("user_history.csv")
user_history_without_user_ID = user_history.drop(['USER ID'],axis=1)
user_history_index = user_history.set_index('USER ID')
user_ratings = pd.read_csv("user_ratings.csv")

In [4]:
# load the divition
train = pd.read_csv("train_rating.csv")
test = pd.read_csv("test_rating.csv")

In [5]:
user_history

,USER ID,alpine kimono,sweden kansas,student icon,supreme ivan,albert charlie,heavy trapeze,fabric tokyo,brother robin,tiger catalog,...,cigar lagoon,equal comedy,bombay podium,helena robot,prodigy rhino,jumbo gray,radius wizard,fame quiz,bazaar complex,glass slogan
0,100950,2.192897,0.361397,0.531663,0.016105,0.694338,1.250662,1.483259,1.799682,0.664616,...,0.309236,2.622033,1.242243,2.947560,0.693319,0.625303,2.352295,1.548417,1.517709,1.508219
1,100956,2.776597,0.788821,1.187149,0.473049,2.245112,1.998881,0.072812,1.441970,2.264368,...,2.304845,1.498307,0.319484,0.089212,3.157167,2.789594,1.003377,1.141516,2.011509,0.377898
2,100962,0.281717,5.046727,4.407484,2.138591,1.075562,0.385842,0.626482,0.026648,1.949374,...,1.463952,0.601814,1.983130,2.364877,0.429133,2.758070,0.563619,0.271453,0.579626,1.785609
3,100969,0.943147,1.165713,2.016138,1.236626,0.973435,2.514205,0.022476,1.091282,1.320748,...,1.428416,3.791742,1.102070,3.250911,1.209403,0.246261,0.558631,1.163652,1.922758,1.008040
4,100974,0.485729,4.633607,4.120416,1.497073,1.463875,0.867737,0.877514,0.019603,1.751483,...,1.179954,1.031741,1.935182,2.012611,0.553173,2.614605,0.313479,0.143246,0.914407,2.011048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,125614,0.200730,3.819307,3.793920,1.352770,1.616516,0.138227,0.193954,0.770098,1.724992,...,0.985254,0.508837,2.033624,2.783638,0.360851,1.787872,0.749357,0.205445,0.945106,1.155107
4496,125622,1.139876,0.361305,1.420300,1.024918,2.313955,1.638803,1.570336,0.233047,1.521743,...,3.606324,1.670899,1.127929,1.888831,4.198021,2.934536,0.501603,0.923039,0.737211,0.675006
4497,125629,2.423679,0.555591,1.244828,0.100391,1.098254,2.027846,1.485342,0.912006,0.304603,...,0.050602,2.439248,1.352124,2.926793,0.254287,0.488679,2.181696,1.367859,1.410247,0.549376
4498,125636,0.096291,3.028528,2.554836,0.404308,1.401119,0.760813,0.356988,1.873021,1.977173,...,0.609813,0.242120,1.061562,2.616126,0.091176,1.162114,1.218459,1.067140,0.441260,0.666655


In [6]:
user_history_index

,alpine kimono,sweden kansas,student icon,supreme ivan,albert charlie,heavy trapeze,fabric tokyo,brother robin,tiger catalog,explain alex,...,cigar lagoon,equal comedy,bombay podium,helena robot,prodigy rhino,jumbo gray,radius wizard,fame quiz,bazaar complex,glass slogan
USER ID,,,,,,,,,,,,,,,,,,,,,
100950,2.192897,0.361397,0.531663,0.016105,0.694338,1.250662,1.483259,1.799682,0.664616,0.165158,...,0.309236,2.622033,1.242243,2.947560,0.693319,0.625303,2.352295,1.548417,1.517709,1.508219
100956,2.776597,0.788821,1.187149,0.473049,2.245112,1.998881,0.072812,1.441970,2.264368,2.518723,...,2.304845,1.498307,0.319484,0.089212,3.157167,2.789594,1.003377,1.141516,2.011509,0.377898
100962,0.281717,5.046727,4.407484,2.138591,1.075562,0.385842,0.626482,0.026648,1.949374,0.062942,...,1.463952,0.601814,1.983130,2.364877,0.429133,2.758070,0.563619,0.271453,0.579626,1.785609
100969,0.943147,1.165713,2.016138,1.236626,0.973435,2.514205,0.022476,1.091282,1.320748,0.525940,...,1.428416,3.791742,1.102070,3.250911,1.209403,0.246261,0.558631,1.163652,1.922758,1.008040
100974,0.485729,4.633607,4.120416,1.497073,1.463875,0.867737,0.877514,0.019603,1.751483,0.297013,...,1.179954,1.031741,1.935182,2.012611,0.553173,2.614605,0.313479,0.143246,0.914407,2.011048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125614,0.200730,3.819307,3.793920,1.352770,1.616516,0.138227,0.193954,0.770098,1.724992,0.550699,...,0.985254,0.508837,2.033624,2.783638,0.360851,1.787872,0.749357,0.205445,0.945106,1.155107
125622,1.139876,0.361305,1.420300,1.024918,2.313955,1.638803,1.570336,0.233047,1.521743,0.799919,...,3.606324,1.670899,1.127929,1.888831,4.198021,2.934536,0.501603,0.923039,0.737211,0.675006
125629,2.423679,0.555591,1.244828,0.100391,1.098254,2.027846,1.485342,0.912006,0.304603,0.219560,...,0.050602,2.439248,1.352124,2.926793,0.254287,0.488679,2.181696,1.367859,1.410247,0.549376


['4.9971665788086455', '2.701128327489425']

In [7]:
learning_rate = 0.0001

def matrixComplete(train, test, user_history = pd.DataFrame({'A' : []})):
    # user_history 
    # user_history need to be PCA before enter the method
    train_ratings = train['RATING']
    train_users = train['USER ID']
    train_movies = train['PRODUCT']
    
    test_ratings = test['RATING']
    test_users = test['USER ID']
    test_movies = test['PRODUCT']
    
    m = len(train_ratings) # the size of the training set
    n_users = train_users.unique().shape[0] # the largest index, plus 1
    n_movies = train_movies.unique().shape[0]
    
    movie_index = pd.unique(train_movies)
    user_index = pd.unique(train_users).sort
    # user_history as rows
    sgd_model = initialize(n_users, n_movies, movie_index, user_index, user_history)
    # user_history appended
    #new = user_history.melt('USER ID', var_name='PRODUCT', value_name='RATING')
    #train_ratings = train_ratings.concat(new['RATING'], axis = 1)
    #train_users = train_users.concat(new['USER ID'], axis = 1)
    #train_movies = train_movies.concat(new['PRODUCT'], axis = 1)
    #sgd_model = initialize(n_users, n_movies)
    
    # epoch is flexible
    epoch = 10
    return train_sgd(sgd_model, epoch, m, train_users, train_movies, train_ratings, 
                     test_users, test_movies, test_ratings, movie_index, user_index, n_movies)

def initialize(n_users, n_movies, movie_index, user_index, rows = pd.DataFrame({'A' : []})):
    """Initalize a random model, and normalize it so that it has sensible mean and variance"""
    # (The normalization helps make sure we start out at a reasonable parameter scale, which speeds up training)
    if (rows.empty):
        # k is flexible
        k = 10
        rows = pd.DataFrame(data = np.random.normal(size=(n_users, k)), index = user_index)
    else:
        k = rows.shape[1]
    
    columns = pd.DataFrame(data = np.random.normal(size=(n_movies, k)), index = movie_index)
    raw_predictions = predict((rows, columns))
    
    s = np.sqrt(2*raw_predictions.std()) # We want to start out with roughly unit variance
    b = np.sqrt((3.5 - raw_predictions.mean()/s)/k) #We want to start out with average rating 3.5
    rows /= s
    rows += b
    columns /= s
    columns += b
    
    return (rows, columns)

def predict(model):
    """The model's predictions for all user/movie pairs"""
    rows, columns = model
    re_rows = rows.to_numpy()
    re_columns = columns.to_numpy()
    return re_rows @ re_columns.T

def single_example_step(model, user, movie, rating):
    """Update the model using the gradient at a single training example"""
    rows, columns = model
    residual = np.dot(rows.loc[user], columns.loc[movie]) - rating
    grad_users = 2 * residual * columns.loc[movie] # the gradient for the rows matrix
    grad_movies = 2 * residual * rows.loc[user] # the gradient for the columns matrix
    rows.loc[user] -= learning_rate*grad_users
    columns.loc[movie] -= learning_rate*grad_movies

def train_sgd(model, epochs, m, train_users, train_movies, train_ratings,
             test_users, test_movies, test_ratings, movie_index, user_index, n_movies):
    """Train the model for a number of epochs via SGD (batch size=1)"""
    rows, columns = model
    # It's good practice to shuffle your data before doing batch gradient descent,
    # so that each mini-batch peforms like a random sample from the dataset
    shuffle = np.random.permutation(m) 
    shuffled_users = train_users[shuffle]
    shuffled_movies = train_movies[shuffle]
    shuffled_ratings = train_ratings[shuffle]
    for epoch in range(epochs):
        for user, movie, rating in zip(shuffled_users, shuffled_movies, shuffled_ratings):
            # update the model using the gradient at a single example
            single_example_step(model, user, movie, rating)
        # after each Epoch, we'll evaluate our model
        predicted = predict(model)
        print(predicted)
        t_pred = pd.DataFrame(predicted.transpose(), index=movie_index).transpose()
        t_pred['USER ID'] = pd.DataFrame(user_index)
        
        print(unindex_pred)
        melt_pred = unindex_pred.melt('USER ID', var_name='PRODUCT', value_name='p_ratings')
        merge_pred_train = train_ratings.to_frame().merge(melt_pred, on = ['USER ID', 'PRODUCT'])
        merge_pred_test = test_ratings.to_frame().merge(melt_pred, on = ['USER ID', 'PRODUCT'])
        
        train_loss = np.mean((merge_pred_train['RATING'] - merge_pred_train['p_ratings'])**2)
        test_loss = np.mean((merge_pred_test['RATING'] - merge_pred_test['p_ratings'])**2)
        print("Loss after epoch #{} is: train/{} --- test/{}".format(epoch+1, train_loss, test_loss))
    return predicted

In [190]:
complete = matrixComplete(train, test, user_history_index)
complete

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


ValueError: DataFrame constructor not properly called!